# Research

In [ ]:
import time
import datetime
import numpy as np
import pandas as pd

from scipy.stats import norm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

from skmultiflow.data import FileStream
from skmultiflow.neural_networks import PerceptronMask

from skfeature.function.similarity_based import lap_score
from skfeature.function.sparse_learning_based import MCFS
from skfeature.utility.construct_W import construct_W

import csv
from csv import DictWriter
import os

from warnings import warn
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
np.seterr(all='ignore')

In [2]:
class FEATURE_WEIGHTING:
    def __init__(self, n_total_ftr, n_selected_features, target_values, model, mu_init=0, sigma_init=1, penalty_s=0.01, penalty_r=0.01, epochs=1, lr_mu=0.01, lr_sigma=0.01, scale_weights=True):
        """
        :param n_total_ftr: (int) Total no. of features
        :param n_selected_ftr: (int) The no. of features to select 
        :param target_values: (np.ndarray) Unique target values (class labels)
        :param mu_init: (int/np.ndarray) Initial importance parameter
        :param sigma_init: (int/np.ndarray) Initial uncertainty parameter
        :param penalty_s: (float) Penalty factor for the uncertainty (corresponds to gamma_s in the paper)
        :param penalty_r: (float) Penalty factor for the regularization (corresponds to gamma_r in the paper)
        :param epochs: (int) No. of epochs that we use each batch of observations to update the parameters
        :param lr_mu: (float) Learning rate for the gradient update of the importance
        :param lr_sigma: (float) Learning rate for the gradient update of the uncertainty
        :param scale_weights: (bool) If True, scale feature weights into the range [0,1]
        :param model: (str) Name of the feature weighting model
        """

        self.n_total_ftr = n_total_ftr
        self.n_selected_ftr = n_selected_features
        self.target_values = target_values
        self.mu = np.ones(n_total_ftr) * mu_init
        self.sigma = np.ones(n_total_ftr) * sigma_init
        self.var = np.ones(n_total_ftr) * mu_init
        self.lap = np.ones(n_total_ftr) * mu_init
        self.mcfs = np.ones(n_total_ftr) * mu_init
        self.penalty_s = penalty_s
        self.penalty_r = penalty_r
        self.epochs = epochs
        self.lr_mu = lr_mu
        self.lr_sigma = lr_sigma
        self.scale_weights = scale_weights
        self.model = model
        self.weights = np.ones(n_total_ftr) * 0
        

        # Additional model-specific parameters
        self.model_param = {}
        
        # FIRES model
        if self.model == 'fires' and tuple(target_values) != (-1, 1):
            if len(np.unique(target_values)) == 2:
                self.model_param['fires'] = True  # Indicates that we need to encode the target variable into {-1,1}
                warn('FIRES WARNING: The target variable will be encoded as: {} = -1, {} = 1'.format(self.target_values[0], self.target_values[1]))
            else:
                raise ValueError('The target variable y must be binary.')
                    
    def weigh_features(self, x, y):
        """
        Compute feature weights, given a batch of observations and corresponding labels
        :param x: (np.ndarray) Batch of observations
        :param y: (np.ndarray) Batch of labels
        :return: feature weights
        :rtype np.ndarray
        """

        # Update estimates of mu and sigma given the predictive model
        if self.model == 'fires':
            self.__fires(x, y)
            
        elif self.model == 'standard':
            self.__standard(x, y)
            
        elif self.model == 'max variance':
            self.__max_variance(x, y)
         
        elif self.model == 'laplacian':
            self.__laplacian_score(x, y)
            
        elif self.model == 'mcfs':
            self.__mcfs_score(x, y)
            
        elif self.model == 'offesel':
            self.__offesel(x, y)
            
        else:
            raise NotImplementedError('The given model name does not exist')

        # Limit sigma to range [0, inf]
        if sum(n < 0 for n in self.sigma) > 0:
            self.sigma[self.sigma < 0] = 0
            warn('Sigma has automatically been rescaled to [0, inf], because it contained negative values.')

        # Compute feature weights
        return self.__compute_weights()

    def __fires(self, x, y):
        """
        Update the distribution parameters mu and sigma by optimizing them in terms of the (log) likelihood.
        Here we assume a Bernoulli distributed target variable. We use a Probit model as our base model.
        This corresponds to the FIRES-GLM model in the paper.
        :param x: (np.ndarray) Batch of observations (numeric values only, consider normalizing data for better results)
        :param y: (np.ndarray) Batch of labels: type binary, i.e. {-1,1} (bool, int or str will be encoded accordingly)
        """

        for epoch in range(self.epochs):
            # Encode target as {-1,1}
            if 'fires' in self.model_param:
                y[y == self.target_values[0]] = -1
                y[y == self.target_values[1]] = 1
                
            # Shuffle the observations
            random_idx = np.random.permutation(len(y))
            x = x[random_idx]
            y = y[random_idx]

            # Iterative update of mu and sigma
            try:
                # Helper functions
                dot_mu_x = np.dot(x, self.mu)
                rho = np.sqrt(1 + np.dot(x**2, self.sigma**2))

                # Gradients
                nabla_mu = norm.pdf(y/rho * dot_mu_x) * (y/rho * x.T)
                nabla_sigma = norm.pdf(y/rho * dot_mu_x) * (- y/(2 * rho**3) * 2 * (x**2 * self.sigma).T * dot_mu_x)

                # Marginal Likelihood
                marginal = norm.cdf(y/rho * dot_mu_x)

                # Update parameters
                self.mu += self.lr_mu * np.mean(nabla_mu / marginal, axis=1)
                self.sigma += self.lr_sigma * np.mean(nabla_sigma / marginal, axis=1)
                    
            except TypeError as e:
                raise TypeError('All features must be a numeric data type.') from e
                
    def __max_variance(self, x, y):
        """ 
        Update the max variance using learning rate.
        Here we calculate the distribution parameters mu and sigma using standard calculation
        
        :param x: (np.ndarray) Batch of observations
        :param y: (np.ndarray) Batch of labels: type binary, i.e. {-1,1} (bool, int or str will be encoded accordingly)
        """
        # Iterative update
        try:
            standardVar = np.var(x, axis = 0)  
            self.var += self.lr_mu * standardVar
 
        except TypeError as e:
            raise TypeError('All features must be a numeric data type.') from e

    def __laplacian_score(self, x, y):
        """ 
        Update the laplacian score using learning rate.
        
        :param x: (np.ndarray) Batch of observations
        :param y: (np.ndarray) Batch of labels: type binary
        """
        # Iterative update
        try:
            standardLap = lap_score.lap_score(X=x, W=construct_W(x))  
            self.lap += self.lr_mu * standardLap
                
        except TypeError as e:
            raise TypeError('All features must be a numeric data type.') from e

    def __mcfs_score(self, x, y):
        """ 
        Update the MCFS score using learning rate.
        
        :param x: (np.ndarray) Batch of observations
        :param y: (np.ndarray) Batch of labels: type binary
        """
        # Iterative update
        try:
            W = MCFS.mcfs(X=x, n_selected_features=self.n_selected_ftr, mode="raw") 
            mcfs_score = W.max(1)
                
            self.mcfs += self.lr_mu * mcfs_score
                
        except TypeError as e:
            raise TypeError('All features must be a numeric data type.') from e
    
    def __offesel(self, x, y):
        """ 
        Update the max mean using learning rate.
        
        :param x: (np.ndarray) Batch of observations
        :param y: (np.ndarray) Batch of labels: type binary
        """
        # Iterative update 
        try:
            standardMean = np.mean(x, axis = 0) 
            self.mu += self.lr_mu * standardMean
    
        except TypeError as e:
            raise TypeError('All features must be a numeric data type.') from e
                

    def __compute_weights(self):
        """
        Compute optimal weights according to the feature weighting model.
        :return: feature weights
        :rtype np.ndarray
        """

        # Compute optimal weights
        if self.model == 'fires':
            weights = (self.mu**2 - self.penalty_s * self.sigma**2) / (2 * self.penalty_r)
            
        elif self.model == 'max variance':
            weights = self.var
         
        elif self.model == 'laplacian':
            weights = self.lap
            
        elif self.model == 'mcfs':
            weights = self.mcfs
            
        elif self.model == 'offesel':
            weights = self.mu
        
        if self.scale_weights:  # Scale weights to [0,1]
            weights = MinMaxScaler().fit_transform(weights.reshape(-1, 1)).flatten()

        return weights

In [3]:
def apply_experiment(dataset_path, dataset_name, tgt_index, batch_sizes, fractions, classifier, model, epochs=1):
    
    final_acc, final_run_time, final_selection_time, final_classification_time = [], [], [], []

    for batch_size in batch_sizes:
        
        if model != 'without fs':
            selected_ftr_lst = fractions
        else:
            selected_ftr_lst = [1]
        
        for selected_ftr in selected_ftr_lst:
            
            acc, fts_time, clf_time = [], [], []
            
            # Load data as scikit-multiflow FileStream
            # NOTE: The models accepts only numeric values. Please one-hot-encode or factorize string/char variables
            # Additionally, we suggest users to normalize all features, e.g. by using scikit-learn's MinMaxScaler()
            stream = FileStream(dataset_path, target_idx=tgt_index)
            stream.prepare_for_use()

            # Initial fit of the predictive model
            predictor = PerceptronMask()
            n_selected_ftr = round(selected_ftr*stream.n_features)
            target_values = stream.target_values

            x, y = stream.next_sample(batch_size=batch_size)
                
            if model != 'without fs':
                start_time_classification = time.time()
                predictor.partial_fit(x, y, target_values)
                clf_time.append(time.time() - start_time_classification)
                
                # Initialize the feature weighting model
                fires_model = FEATURE_WEIGHTING(
                              n_total_ftr=stream.n_features,          # Total no. of features
                              n_selected_features = n_selected_ftr,   # no. of selected features
                              target_values=target_values,            # Unique target values (class labels)
                              mu_init=0,                              # Initial importance parameter
                              sigma_init=1,                           # Initial uncertainty parameter
                              penalty_s=0.01,                         # Penalty factor for the uncertainty (corresponds to gamma_s in the paper)
                              penalty_r=0.01,                         # Penalty factor for the regularization (corresponds to gamma_r in the paper)
                              epochs=epochs,                          # No. of epochs that we use each batch of observations to update the parameters
                              lr_mu=0.01,                             # Learning rate for the gradient update of the importance
                              lr_sigma=0.01,                          # Learning rate for the gradient update of the uncertainty
                              scale_weights=True,                     # If True, scale feature weights into the range [0,1]
                              model=model)                            # Name of the feature weighting model
            else:
                start_time_classification = time.time()
                predictor.partial_fit(x, y, target_values)
                clf_time.append(time.time() - start_time_classification)
   
            while stream.has_more_samples():
                # Load a new sample
                x, y = stream.next_sample(batch_size=batch_size)
                
                # Normalize using MinMaxScaler
                scaler = MinMaxScaler()
                x = scaler.fit_transform(x)
                
                if model != 'without fs':
                
                    # Select features
                    start_time_selection = time.time()
                    ftr_weights = fires_model.weigh_features(x, y)  # Get feature weights
                    fts_time.append(time.time() - start_time_selection)
                    
                    if model == 'mcfs':
                        ftr_selection = np.argsort(ftr_weights,0)[::-1][:n_selected_ftr]
                    elif model == 'laplacian':
                        ftr_selection = np.argsort(ftr_weights,0)[:n_selected_ftr]
                    else:
                        ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]
                        

                    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
                    x_reduced = np.zeros(x.shape)
                    x_reduced[:, ftr_selection] = x[:, ftr_selection]
                    
        
                    # Test
                    y_pred = predictor.predict(x_reduced)
                    acc_score = accuracy_score(y, y_pred)
                    acc.append(acc_score)
                    
                    # Train
                    start_time_classification = time.time()
                    predictor.partial_fit(x_reduced, y)
                    clf_time.append(time.time() - start_time_classification)
                    
                       
                else:
                    fts_time.append(0)
                    
                    # Test
                    y_pred = predictor.predict(x)
                    acc_score = accuracy_score(y, y_pred)
                    acc.append(acc_score)
        
                    # Train
                    start_time_classification = time.time()
                    predictor.partial_fit(x, y)
                    clf_time.append(time.time() - start_time_classification)
                
            
            # Average accuracy, feature selection time and training time
            final_acc.append(np.mean(acc))
            final_selection_time.append(np.mean(fts_time))
            final_classification_time.append(np.mean(clf_time))
        
            # Restart the FileStream
            stream.restart()
            
    avg_acc_score = np.mean(final_acc) 
    avg_clf_time_score = np.mean(final_classification_time) * 1000
    avg_fts_time_score = np.mean(final_selection_time) * 1000
    avg_time_score = avg_clf_time_score + avg_fts_time_score
    
    summary = {
        'Timestamp': datetime.datetime.now(),
        'Dataset Name': dataset_name,
        'Model Name': model,
        'Classifier': classifier, 
        'Accuracy': round(avg_acc_score,3),
        'Computation Time': round(avg_time_score,1), 
        'Feature Selection Time': round(avg_fts_time_score,1), 
        'Training Time': round(avg_clf_time_score,1)
    }

    return summary
    
    

# Experiments

In [ ]:
# Define the expirements (feature selction model and classifier)
expirements = {
    'FIRES_perceptron': ['perceptron', 'fires'],
    'Max_Variance_perceptron': ['perceptron', 'max variance'],
    'Laplacian_Score_perceptron': ['perceptron', 'laplacian'],
    'MCFS_Score_perceptron': ['perceptron', 'mcfs'],
    'OFFESEL_perceptron': ['perceptron', 'offesel'],
    'Without_Feature_Selection_perceptron' : ['perceptron', 'without fs'],  
}

# Define the paths of the datasets
datasets_info = {
    "HAR":["datasets/har/har_iqr.csv", -1, 2],
    "Spambase":["datasets/spambase/spambase_iqr.csv", -1, 1],
    "Gisette":["datasets/gisette/gisette.csv", -1, 5],
    "Madelon":["datasets/madelon/madelon_iqr.csv",-1, 5],
    "Dota":["datasets/dota/dota.csv", -1, 5],
    "KDD":["datasets/kdd/kdd_iqr_binary_encoded_categorical.csv", -1, 1],
    "MNIST":["datasets/mnist/mnist.csv", -1, 1],
    "RBF":["datasets/rbf/rbf.csv", -1, 5],
    "RTG":["datasets/rtg/rtg.csv", -1, 1],
    "Forest CoverType":["datasets/covertype/covtype_iqr.csv",-1, 1],
    "Poker Hand":["datasets/poker/poker_hand_encoded_categorical.csv", -1, 1],
    "Electricity":["datasets/electricity/electricity.csv",-1, 1],
    "Airlines":["datasets/airlines/airlines_iqr_encoded_categorical.csv",-1, 1],
    "10 KDD99":["datasets/10kdd/10kdd_iqr_binary_encoded_categorical.csv", -1, 1],
    "Spam Assassin Corpus":["datasets/spam/spam_corpus.csv", -1, 1],
    "Usenet":["datasets/usenet/usenet.csv", -1, 1],
    "Sensor Stream":["datasets/sensor/sensor_iqr.csv", -1, 1],
    "Energy Grids":["datasets/energygrids/energy_grids.csv", -1,1]
}

     
# Define the filename of the CSV results file
filename = 'results.csv'

# Define the headers for the CSV results file
headers = ['Timestamp', 'Dataset Name', 'Model Name', 'Classifier', 'Accuracy','Computation Time', 'Feature Selection Time', 'Training Time']

# Check if the CSV results file exists
if not os.path.exists(filename):
    # If the file does not exist, create it with headers
    with open(filename, 'w', encoding='UTF8', newline='') as file:
        # Create a CSV writer object
        dictwriter_object = DictWriter(file, fieldnames=headers)
        # Write the headers names
        dictwriter_object.writeheader()
        
    #Close the file object
    file.close()
    
    
for dataset_name, dataset_params in datasets_info.items():
    for exp, exp_params in expirements.items():
        
        print('#'*50)

        summary = apply_experiment(dataset_path=dataset_params[0], dataset_name=dataset_name, tgt_index=dataset_params[1], epochs=dataset_params[2], batch_sizes=[25, 50, 75, 100], fractions=[0.1, 0.15, 0.2], classifier=exp_params[0], model=exp_params[1])
        
        # Print the summary values
        result_string = f"{summary['Timestamp']} - Dataset: {summary['Dataset Name']}, Model: {summary['Model Name']}, Accuracy: {summary['Accuracy']}, Computation Time: {summary['Computation Time']}, Feature Selection Time: {summary['Feature Selection Time']}, Training Time: {summary['Training Time']}"
        
        print(result_string)
        
        with open(filename, 'a', encoding='UTF8', newline='') as file:
            # Create a CSV writer object
            dictwriter_object = DictWriter(file, fieldnames=headers)
            # Write a new row of data to the CSV file
            dictwriter_object.writerow(summary)
  
        #Close the file object
        file.close()